In [1]:
# from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH

from typing import Optional

# path to the SDK folder
SDK_PATH: Optional[str] = '/home/viki/Documents/BSc-Thesis/CMU-MultimodalSDK'
print("SDK_PATH:", SDK_PATH)

# path to the folder where you want to store data
DATA_PATH: Optional[str] = '/home/viki/Documents/BSc-Thesis/Thesis/data'

# path to a pretrained word embedding file
WORD_EMB_PATH: Optional[str] = '/home/viki/Documents/BSc-Thesis/Thesis/data/http__immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWordVectors.csd'

# path to loaded word embedding matrix and corresponding word2id mapping
CACHE_PATH: Optional[str] = None
# CACHE_PATH: Optional[str] = 'C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data\\embedding_and_mapping.pt'



import sys
import pandas as pd

SDK_PATH: /home/viki/Documents/BSc-Thesis/CMU-MultimodalSDK


In [2]:


if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)
    print(f"SDK path is set to {SDK_PATH}")

SDK path is set to /home/viki/Documents/BSc-Thesis/CMU-MultimodalSDK


In [3]:
import sys
for path in sys.path:
    print(path)
    

/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/viki/Documents/BSc-Thesis/.venv/lib/python3.10/site-packages
/home/viki/Documents/BSc-Thesis/CMU-MultimodalSDK


In [4]:
import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError



import csv


# we can see they are in the format of 'video_id[segment_no]', but the splits was specified with video_id only
# we need to use regex or something to match the video IDs...
import torch
import torch.nn as nn

from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook
from collections import defaultdict
import random

from torch.optim import Adam, SGD
from sklearn.metrics import accuracy_score
import tqdm
from tqdm import tqdm_notebook
import random
import numpy as np


from torch import optim
# from torch.utils.tensorboard import SummaryWriter





In [5]:

if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

DATASET = md.cmu_mosi

try:
    md.mmdataset(DATASET.highlevel, DATA_PATH)
except RuntimeError:
    print("High-level features have been downloaded previously.")

try:
    md.mmdataset(DATASET.raw, DATA_PATH)
except RuntimeError:
    print("Raw data have been downloaded previously.")
    
try:
    md.mmdataset(DATASET.labels, DATA_PATH)
except RuntimeError:
    print("Labels have been downloaded previously.")

[2024-12-11 17:06:09.941] | Error   | /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_TimestampedWordVectors.csd file already exists ...
High-level features have been downloaded previously.
[2024-12-11 17:06:09.942] | Error   | /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_TimestampedWords.csd file already exists ...
Raw data have been downloaded previously.
[2024-12-11 17:06:09.942] | Error   | /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_Opinion_Labels.csd file already exists ...
Labels have been downloaded previously.


In [ ]:
# list the directory contents... let's see what features there are
data_files = os.listdir(DATA_PATH)
print('\n'.join(data_files))

embedding_and_mapping.pt
http__immortal.multicomp.cs.cmu.edu


In [ ]:
import os

folder_path = "C:\\Users\\Viki\\Documents\\Thesis\\tryout3\\data"
for root, dirs, files in os.walk(folder_path):
    print(f"Directory: {root}")
    for file in files:
        print(f"  {file}")


Directory: C:\Users\Viki\Documents\Thesis\tryout3\data
  embedding_and_mapping.pt
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\acoustic
  CMU_MOSI_COVAREP.csd
  CMU_MOSI_OpenSmile_EB10.csd
  CMU_MOSI_openSMILE_IS09.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\labels
  CMU_MOSI_Opinion_Labels.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\language
  CMU_MOSI_TimestampedPhones.csd
  CMU_MOSI_TimestampedWords.csd
  CMU_MOSI_TimestampedWordVectors.csd
Directory: C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\visual
  CMU_MOSI_Visual_Facet_41.csd
  CMU_MOSI_Visual_Facet_42.csd
  CMU_MOSI_Visual

In [7]:
# Define your different modalities - refer to the filenames of the CSD files
visual_field_Facet41 = 'CMU_MOSI_Visual_Facet_41'
visual_field_Facet42 = 'CMU_MOSI_Visual_Facet_42'
visual_field_OpenFace1 = 'CMU_MOSI_Visual_OpenFace_1'


# visual_field_OpenFace2 = 'CMU_MOSI_Visual_OpenFace_2'
# [2024-11-24 21:59:03.886] | Error   | C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\visual\CMU_MOSI_Visual_OpenFace_2.csd resource is not a valid hdf5 computational sequence format ...


acoustic_field_COVAREP = 'CMU_MOSI_COVAREP'
acoustic_field_OpenSmile_EB10 = 'CMU_MOSI_OpenSmile_EB10'
acoustic_field_OpenSmile_IS09 = 'CMU_MOSI_openSMILE_IS09'



text_field_Words = 'CMU_MOSI_TimestampedWords'
text_field_Phones = 'CMU_MOSI_TimestampedPhones'
text_field_WordVectors = 'CMU_MOSI_TimestampedWordVectors'




In [11]:

# text_field = 'CMU_MOSI_ModifiedTimestampedWords'


# List of features
features = [
    text_field_Words,
    # text_field_Phones,
    # text_field_WordVectors,
    visual_field_Facet41, 
    # visual_field_Facet42,
    # visual_field_OpenFace1,
    # visual_field_OpenFace2,
    acoustic_field_COVAREP,
    # acoustic_field_OpenSmile_IS09,
    # acoustic_field_OpenSmile_EB10
]


# Recipe with correct subdirectory paths for each modality
recipe = {
    # text_field_Words: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "language", text_field_Words) + '.csd',
    text_field_Words: os.path.join(DATA_PATH, text_field_Words) + '.csd',


    # not helpful
    # text_field_Phones: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu","CMU-MOSI", "language", text_field_Phones) + '.csd',

    # text_field_WordVectors: os.path.join(DATA_PATH,  text_field_WordVectors) + '.csd',
    
    visual_field_Facet41: os.path.join(DATA_PATH, visual_field_Facet41) + '.csd',
    
    # visual_field_Facet42: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_Facet42) + '.csd',
    
    # visual_field_OpenFace1: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_OpenFace1) + '.csd',
    
    
    #not taken into account cause not runnable - 
    # visual_field_OpenFace2: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "visual", visual_field_OpenFace2) + '.csd',
    
    acoustic_field_COVAREP: os.path.join(DATA_PATH, acoustic_field_COVAREP) + '.csd',
    
    # acoustic_field_OpenSmile_EB10: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "acoustic", acoustic_field_OpenSmile_EB10) + '.csd',
    
    # has same and less features as EB10
    # acoustic_field_OpenSmile_IS09: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "acoustic", acoustic_field_OpenSmile_IS09) + '.csd'
}



# print (recipe)
dataset = md.mmdataset(recipe)

[2024-12-11 17:08:31.145] | Success | Computational sequence read from file /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_TimestampedWords.csd ...
[2024-12-11 17:08:31.192] | Status  | Checking the integrity of the <words> computational sequence ...
[2024-12-11 17:08:31.192] | Status  | Checking the format of the data in <words> computational sequence ...


  0%|          | 0/93 [00:00<?, ? Computational Sequence Entries/s]

[2024-12-11 17:08:31.364] | Success | <words> computational sequence data in correct format.
[2024-12-11 17:08:31.364] | Status  | Checking the format of the metadata in <words> computational sequence ...
[2024-12-11 17:08:31.365] | Warning | <words> computational sequence does not have all the required metadata ... continuing 
[2024-12-11 17:08:31.370] | Success | Computational sequence read from file /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_Visual_Facet_41.csd ...
[2024-12-11 17:08:31.433] | Status  | Checking the integrity of the <FACET_4.1> computational sequence ...
[2024-12-11 17:08:31.433] | Status  | Checking the format of the data in <FACET_4.1> computational sequence ...


[2024-12-11 17:08:31.740] | Success | <FACET_4.1> computational sequence data in correct format.
[2024-12-11 17:08:31.740] | Status  | Checking the format of the metadata in <FACET_4.1> computational sequence ...
[2024-12-11 17:08:31.740] | Warning | <FACET_4.1> computational sequence does not have all the required metadata ... continuing 
[2024-12-11 17:08:31.747] | Success | Computational sequence read from file /home/viki/Documents/BSc-Thesis/Thesis/data/CMU_MOSI_COVAREP.csd ...
[2024-12-11 17:08:31.798] | Status  | Checking the integrity of the <COVAREP> computational sequence ...
[2024-12-11 17:08:31.798] | Status  | Checking the format of the data in <COVAREP> computational sequence ...


[2024-12-11 17:08:31.937] | Success | <COVAREP> computational sequence data in correct format.
[2024-12-11 17:08:31.937] | Status  | Checking the format of the metadata in <COVAREP> computational sequence ...
[2024-12-11 17:08:31.938] | Warning | <COVAREP> computational sequence does not have all the required metadata ... continuing 
[2024-12-11 17:08:31.938] | Success | Dataset initialized successfully ... 


In [12]:

print("=== Metadata FACE 41 Visual===\n")
print("Visual FACE 41 Metadata:", dataset[visual_field_Facet41].metadata)
print("\n")

print("=== Metadata COVAREP Acoustic===\n")
print("Acoustic COVAREP Metadata:", dataset[acoustic_field_COVAREP].metadata)
print("\n")

print("=== Metadata Words===\n")
print("Words Metadata:", dataset[text_field_Words].metadata)
print("\n")

print("=== Metadata WordVectors===\n")
print("WordVectors Metadata:", dataset[text_field_WordVectors].metadata)
print("\n")


=== Metadata FACE 41 Visual===

Visual FACE 41 Metadata: {'alignment compatible': True, 'computational sequence description': 'FACET 4.1 Visual Features for CMU-MOSI Dataset', 'computational sequence version': 1.0, 'contact': 'abagherz@andrew.cmu.edu', 'creator': 'Amir Zadeh', 'dataset bib citation': '@article{zadeh2016multimodal,title={Multimodal sentiment intensity analysis in videos: Facial gestures and verbal messages},author={Zadeh, Amir and Zellers, Rowan and Pincus, Eli and Morency, Louis-Philippe},journal={IEEE Intelligent Systems},volume={31},number={6},pages={82--88},year={2016},publisher={IEEE}}', 'dataset name': 'CMU-MOSI', 'dataset version': 1.0, 'dimension names': ['Face X', 'Face Y', 'Face Width', 'Face Height', 'angerEvidence', 'contemptEvidence', 'disgustEvidence', 'joyEvidence', 'fearEvidence', 'negativeEvidence', 'neutralEvidence', 'positiveEvidence', 'sadnessEvidence', 'surpriseEvidence', 'confusionEvidence', 'frustrationEvidence', 'angerIntensity', 'contemptIntensi

RuntimeError: Computational sequence does not exist ...

In [ ]:
fields_metadata = [
    {"Field": "Visual FACE 41", "Metadata": dataset[visual_field_Facet41].metadata},
    # {"Field": "Visual FACE 42", "Metadata": dataset[visual_field_Facet42].metadata},
    # {"Field": "Visual OpenFace", "Metadata": dataset[visual_field_OpenFace1].metadata},
    {"Field": "Acoustic COVAREP", "Metadata": dataset[acoustic_field_COVAREP].metadata},
    # {"Field": "Acoustic OpenSmile EB10", "Metadata": dataset[acoustic_field_OpenSmile_EB10].metadata},
    # {"Field": "Acoustic OpenSmile IS09", "Metadata": dataset[acoustic_field_OpenSmile_IS09].metadata},
    {"Field": "Words", "Metadata": dataset[text_field_Words].metadata},
    # {"Field": "Phones", "Metadata": dataset[text_field_Phones].metadata},
    {"Field": "WordVectors", "Metadata": dataset[text_field_WordVectors].metadata},
]

# Normalize (flatten) the metadata dictionaries
normalized_data = []
for entry in fields_metadata:
    metadata_flat = pd.json_normalize(entry["Metadata"], sep="_")
    metadata_flat["Field"] = entry["Field"]
    normalized_data.append(metadata_flat)

# print(f"normalized metadata: {normalized_data}")

df_metadata = pd.concat(normalized_data, ignore_index=True)
df_metadata = df_metadata[["Field"] + [col for col in df_metadata.columns if col != "Field"]]

print(df_metadata)



# # Save the DataFrame to a CSV file
# csv_filename = "metadata_summary.csv"
# df_metadata.to_csv(csv_filename, index=False)
# 
# print(f"Metadata successfully saved to {csv_filename}")
# 
# 
# 
#remove \ for Acoustic OpenSmile EB10
#remove b" and " for Acoustic OpenSmile IS09

              Field  alignment compatible  \
0    Visual FACE 41                  True   
1  Acoustic COVAREP                  True   
2             Words                 False   
3       WordVectors                  True   

               computational sequence description  \
0  FACET 4.1 Visual Features for CMU-MOSI Dataset   
1  COVAREP Acoustic Features for CMU-MOSI Dataset   
2             Word sequences for CMU-MOSI Dataset   
3      Word vector sequences for CMU-MOSI Dataset   

   computational sequence version                  contact     creator  \
0                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
1                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
2                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   
3                             1.0  abagherz@andrew.cmu.edu  Amir Zadeh   

                                dataset bib citation dataset name  \
0  @article{zadeh2016multimodal,title={Multimodal...     CMU-MOSI   

In [17]:
df_components = df_metadata[["Field", "dimension names"]]

# Define the replacements in a dictionary
replacements = {
    'b"': '',
    '"': '',
    '\'' : '',
    '[': '',
    ']': ''

}



def replace_strings(x):
    if isinstance(x, str):
        for old, new in replacements.items():
            x = x.replace(old, new)
    return x


# Apply the replacements using map
df_components = df_components.applymap(replace_strings)

print(df_components)

# # Save the DataFrame to a CSV file
# csv_filename = "componentsWithoutIS.csv"
# df_components.to_csv(csv_filename, index=False)

NameError: name 'df_metadata' is not defined

In [20]:
# import pandas as pd
# import csv

# # Define the replacements in a dictionary
# replacements = {
#     'b"': '',
#     '\"': '',
#     '\'': '',
#     '[': '',
#     ']': ''
# }

# # Function to apply replacements
# def replace_strings(x):
#     if isinstance(x, str):
#         for old, new in replacements.items():
#             x = x.replace(old, new)
#     return x

# # Read the CSV file into a DataFrame
# with open('componentsWithoutIS.csv', 'r', newline='', encoding='utf-8') as csvfile:
#     reader = csv.reader(csvfile)
#     rows = []
#     for row in reader:
#         new_row = []
#         for element in row:
#             # Split elements by comma and strip spaces
#             split_elements = [e.strip() for e in element.split(',')]
#             new_row.extend(split_elements)
#         rows.append(new_row)

# # Create DataFrame from the processed rows
# df = pd.DataFrame(rows)

# # Apply the replacements using applymap to the entire DataFrame
# df = df.applymap(replace_strings)

# # # Save the modified DataFrame back to a CSV file
# # df.to_csv('componentsModifiedWithoutIS.csv', index=False)

In [ ]:

# # Load the dataset from a CSV file
# # Replace 'your_file.csv' with the path to your actual file
# df = pd.read_csv('componentsModifiedWithoutIS.csv', header=None)

# # Flatten the DataFrame into a single list, keeping track of field positions
# data = []
# for col in df.columns:
#     for row in df.index:
#         value = df.iloc[row, col]
#         if pd.notna(value):  # Exclude empty elements
#             data.append((value.strip(), f"Row {row+1}, Column {col+1}"))

# # Create a dictionary to count occurrences and track positions
# duplicate_tracker = {}
# for value, position in data:
#     if value in duplicate_tracker:
#         duplicate_tracker[value]['count'] += 1
#         duplicate_tracker[value]['positions'].append(position)
#     else:
#         duplicate_tracker[value] = {'count': 1, 'positions': [position]}

# # Write results to a text file
# with open('outputDuplicatesWithoutIS.txt', 'w') as output_file:
#     output_file.write("Duplicates Found:\n")
#     for value, info in duplicate_tracker.items():
#         if info['count'] > 1:
#             output_file.write(
#                 f"{value}: {info['count']}; Positions: {', '.join(info['positions'])}\n"
#             )

# print("Duplicate analysis complete. Results saved to 'outputWithoutIS.txt'.")


FileNotFoundError: [Errno 2] No such file or directory: 'componentsModifiedWithoutIS.csv'

In [ ]:
# we define a simple averaging function that does not depend on intervals
def avg(intervals: np.array, features: np.array) -> np.array:
    try:
        return np.average(features, axis=0)
    except:
        return features

# first we align to words with averaging, collapse_function receives a list of functions
dataset.align(text_field_Words, collapse_functions=[avg])

[2024-12-11 17:10:57.595] | Status  | Unify was called ...
[2024-12-11 17:10:57.596] | Success | Unify completed ...
[2024-12-11 17:10:57.596] | Status  | Pre-alignment based on <CMU_MOSI_TimestampedWords> computational sequence started ...
[2024-12-11 17:11:06.323] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2024-12-11 17:11:08.355] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2024-12-11 17:11:08.375] | Status  | Alignment starting ...


Overall Progress:  52%|█████▏    | 48/93 [01:52<01:36,  2.13s/ Computational Sequence Entries]

KeyboardInterrupt: 

In [ ]:
label_field = 'CMU_MOSI_Opinion_Labels'

# we add and align to lables to obtain labeled segments
# this time we don't apply collapse functions so that the temporal sequences are preserved
label_recipe = {label_field: os.path.join(DATA_PATH, "http__immortal.multicomp.cs.cmu.edu", "CMU-MOSI", "labels", label_field) + '.csd'}
dataset.add_computational_sequences(label_recipe, destination=None)
dataset.align(label_field)

[2024-12-04 11:09:47.636] | Success | Computational sequence read from file C:\Users\Viki\Documents\Thesis\tryout3\data\http__immortal.multicomp.cs.cmu.edu\CMU-MOSI\labels\CMU_MOSI_Opinion_Labels.csd ...
[2024-12-04 11:09:47.642] | Status  | Checking the integrity of the <Opinion Segment Labels> computational sequence ...
[2024-12-04 11:09:47.642] | Status  | Checking the format of the data in <Opinion Segment Labels> computational sequence ...


[2024-12-04 11:09:47.682] | Success | <Opinion Segment Labels> computational sequence data in correct format.
[2024-12-04 11:09:47.682] | Status  | Checking the format of the metadata in <Opinion Segment Labels> computational sequence ...
[2024-12-04 11:09:47.682] | Warning | <Opinion Segment Labels> computational sequence does not have all the required metadata ... continuing 
[2024-12-04 11:09:47.682] | Status  | Unify was called ...
[2024-12-04 11:09:47.804] | Success | Unify completed ...
[2024-12-04 11:09:47.807] | Status  | Pre-alignment based on <CMU_MOSI_Opinion_Labels> computational sequence started ...
[2024-12-04 11:09:47.935] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWords> ...
[2024-12-04 11:09:48.192] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWordVectors> ...
[2024-12-04 11:09:48.345] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2024-12-04 11:09:48.484] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2024-12-04 

Aligning 03bSnISJMiM:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                   
Aligning 0h-zjBukYpk:   0%|          | 0/25 [00:00<?, ? Segments/s]
                                                                   
Aligning 1DmNV9C1hbY:   0%|          | 0/14 [00:00<?, ? Segments/s]
                                                                   
Aligning 1iG0909rllw:   0%|          | 0/30 [00:00<?, ? Segments/s]
                                                                   
Aligning 2iD-tVS8NPw:   0%|          | 0/30 [00:00<?, ? Segments/s]
                                                                   
Aligning 5W7Z1C_fDaE:   0%|          | 0/24 [00:00<?, ? Segments/s]
                                                                   
Aligning 6Egk_28TtTM:   0%|          | 0/12 [00:00<?, ? Segments/s]
                                                                   
Aligning 6_0THN4chvY:   0%|          | 0/14 [00:

[2024-12-04 11:09:50.930] | Success | Alignment to <CMU_MOSI_Opinion_Labels> complete.
[2024-12-04 11:09:50.931] | Status  | Replacing dataset content with aligned computational sequences
[2024-12-04 11:09:51.081] | Success | Initialized empty <CMU_MOSI_TimestampedWords> computational sequence.
[2024-12-04 11:09:51.081] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWords> computational sequence ...


[2024-12-04 11:09:51.087] | Success | <CMU_MOSI_TimestampedWords> computational sequence data in correct format.
[2024-12-04 11:09:51.087] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWords> computational sequence ...
[2024-12-04 11:09:51.087] | Warning | <CMU_MOSI_TimestampedWords> computational sequence does not have all the required metadata ... continuing 
[2024-12-04 11:09:51.087] | Success | Initialized empty <CMU_MOSI_TimestampedWordVectors> computational sequence.
[2024-12-04 11:09:51.087] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWordVectors> computational sequence ...


[2024-12-04 11:09:51.093] | Success | <CMU_MOSI_TimestampedWordVectors> computational sequence data in correct format.
[2024-12-04 11:09:51.093] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWordVectors> computational sequence ...
[2024-12-04 11:09:51.093] | Warning | <CMU_MOSI_TimestampedWordVectors> computational sequence does not have all the required metadata ... continuing 
[2024-12-04 11:09:51.093] | Success | Initialized empty <CMU_MOSI_Visual_Facet_41> computational sequence.
[2024-12-04 11:09:51.093] | Status  | Checking the format of the data in <CMU_MOSI_Visual_Facet_41> computational sequence ...


[2024-12-04 11:09:51.098] | Success | <CMU_MOSI_Visual_Facet_41> computational sequence data in correct format.
[2024-12-04 11:09:51.098] | Status  | Checking the format of the metadata in <CMU_MOSI_Visual_Facet_41> computational sequence ...
[2024-12-04 11:09:51.098] | Warning | <CMU_MOSI_Visual_Facet_41> computational sequence does not have all the required metadata ... continuing 
[2024-12-04 11:09:51.098] | Success | Initialized empty <CMU_MOSI_COVAREP> computational sequence.
[2024-12-04 11:09:51.098] | Status  | Checking the format of the data in <CMU_MOSI_COVAREP> computational sequence ...


[2024-12-04 11:09:51.105] | Success | <CMU_MOSI_COVAREP> computational sequence data in correct format.
[2024-12-04 11:09:51.105] | Status  | Checking the format of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
[2024-12-04 11:09:51.105] | Warning | <CMU_MOSI_COVAREP> computational sequence does not have all the required metadata ... continuing 
[2024-12-04 11:09:51.105] | Success | Initialized empty <CMU_MOSI_Opinion_Labels> computational sequence.
[2024-12-04 11:09:51.105] | Status  | Checking the format of the data in <CMU_MOSI_Opinion_Labels> computational sequence ...


[2024-12-04 11:09:51.112] | Success | <CMU_MOSI_Opinion_Labels> computational sequence data in correct format.
[2024-12-04 11:09:51.112] | Status  | Checking the format of the metadata in <CMU_MOSI_Opinion_Labels> computational sequence ...
[2024-12-04 11:09:51.112] | Warning | <CMU_MOSI_Opinion_Labels> computational sequence does not have all the required metadata ... continuing 


In [ ]:
num_segments = len(dataset[visual_field_Facet41].keys())  # Assuming all fields have the same segment keys
print(f"Number of segments: {num_segments}")


Number of segments: 2198


In [ ]:
# Extract unique video IDs
video_ids = set(seg.split("[")[0] for seg in dataset[visual_field_Facet41].keys())

# Count the number of unique video IDs
num_unique_ids = len(video_ids)

# Count segments for each video ID
video_segment_counts = {video_id: sum(seg.startswith(video_id) for seg in dataset[visual_field_Facet41].keys()) for video_id in video_ids}

# Print the total number of unique video IDs
print(f"Total number of unique video IDs: {num_unique_ids}\n")

# Print the segment counts for each video ID
for video_id, count in video_segment_counts.items():
    print(f"Video {video_id}: {count} segments")



Total number of unique video IDs: 93

Video G-xst2euQUc: 21 segments
Video vyB00TXsimI: 22 segments
Video Vj1wYRQjB-o: 9 segments
Video atnd_PF-Lbs: 21 segments
Video iiK8YX8oH1E: 18 segments
Video cXypl4FnoZo: 29 segments
Video LSi-o-IrDMs: 29 segments
Video c5xsKMxpXnc: 15 segments
Video 9c67fiY0wGQ: 12 segments
Video Jkswaaud0hk: 20 segments
Video 5W7Z1C_fDaE: 24 segments
Video bvLlb-M3UXU: 25 segments
Video v0zCBqDeKcE: 16 segments
Video Qr1Ca94K55A: 14 segments
Video Nzq88NnDkEk: 32 segments
Video BioHAh1qJAQ: 30 segments
Video QN9ZIUWUXsY: 18 segments
Video IumbAb8q2dM: 22 segments
Video 6_0THN4chvY: 14 segments
Video BvYR0L6f2Ig: 26 segments
Video OtBXNcAL_lE: 24 segments
Video BI97DNYfe5I: 31 segments
Video tIrG4oNLFzE: 18 segments
Video 7JsX8y1ysxY: 39 segments
Video 8OtFthrtaJM: 23 segments
Video f9O3YtZ2VfI: 30 segments
Video 8qrpnFRGt2A: 26 segments
Video ZAIRrfG22O0: 9 segments
Video 03bSnISJMiM: 13 segments
Video GWuJjcEuzt8: 18 segments
Video PZ-lDQFboO8: 12 segments
Vid

In [ ]:
# obtain the train/dev/test splits - these splits are based on video IDs
train_split = DATASET.standard_folds.standard_train_fold
dev_split = DATASET.standard_folds.standard_valid_fold
test_split = DATASET.standard_folds.standard_test_fold

# inspect the splits: they only contain video IDs
print(f"lengths: train {len(train_split)}, dev {len(dev_split)}, test {len(test_split)}\n")
print(train_split)
print(dev_split)
print(test_split)

lengths: train 52, dev 10, test 31

['2iD-tVS8NPw', '8d-gEyoeBzc', 'Qr1Ca94K55A', 'Ci-AH39fi3Y', '8qrpnFRGt2A', 'Bfr499ggo-0', 'QN9ZIUWUXsY', '9T9Hf74oK10', '7JsX8y1ysxY', '1iG0909rllw', 'Oz06ZWiO20M', 'BioHAh1qJAQ', '9c67fiY0wGQ', 'Iu2PFX3z_1s', 'Nzq88NnDkEk', 'Clx4VXItLTE', '9J25DZhivz8', 'Af8D0E4ZXaw', 'TvyZBvOMOTc', 'W8NXH0Djyww', '8OtFthrtaJM', '0h-zjBukYpk', 'Vj1wYRQjB-o', 'GWuJjcEuzt8', 'BI97DNYfe5I', 'PZ-lDQFboO8', '1DmNV9C1hbY', 'OQvJTdtJ2H4', 'I5y0__X72p0', '9qR7uwkblbs', 'G6GlGvlkxAQ', '6_0THN4chvY', 'Njd1F0vZSm4', 'BvYR0L6f2Ig', '03bSnISJMiM', 'Dg_0XKD0Mf4', '5W7Z1C_fDaE', 'VbQk4H8hgr0', 'G-xst2euQUc', 'MLal-t_vJPM', 'BXuRRbG0Ugk', 'LSi-o-IrDMs', 'Jkswaaud0hk', '2WGyTLYerpo', '6Egk_28TtTM', 'Sqr0AcuoNnk', 'POKffnXeBds', '73jzhE8R1TQ', 'OtBXNcAL_lE', 'HEsqda8_d0Q', 'VCslbP0mgZI', 'IumbAb8q2dM']
['WKA5OygbEKI', 'c5xsKMxpXnc', 'atnd_PF-Lbs', 'bvLlb-M3UXU', 'bOL9jKpeJRs', '_dI--eQ6qVU', 'ZAIRrfG22O0', 'X3j2zQgwYgE', 'aiEXnCPZubE', 'ZUXBRvtny7o']
['tmZoasNr4rU', 'zhpQhgha_KU', '

In [ ]:

# A sentinel epsilon for safe division, avoiding division by zero
EPS = 1e-8

word2id = defaultdict(lambda: len(word2id))
UNK = word2id['<unk>']
PAD = word2id['<pad>']
# EOS = word2id['<eos>']
# BOS = word2id['<bos>']
# # SEP = word2id['<sep>']
# DUMMY = word2id['<dummy>']

pattern = re.compile('(.*)\[.*\]')

In [ ]:
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")


# Assuming the input features are already defined elsewhere
# _words, _visual_Facet41, _visual_Facet42, _visual_OpenFace1, _acoustic_COVAREP, _acoustic_OpenSmile_EB10, _wordvectors
# Also assuming train_split, dev_split, test_split are already defined

# Placeholders for final train/dev/test dataset
train = []
dev = []
test = []

# Iterate over the segments in the dataset
num_drop = 0  # Counter to track the number of dropped data points

for segment in dataset[label_field].keys():
    # Get the video ID and features
    vid = re.search(pattern, segment).group(1)
    label = dataset[label_field][segment]['features']
    _words = dataset[text_field_Words][segment]['features']

    # Collect all visual and acoustic features
    _visual_Facet41 = dataset[visual_field_Facet41][segment]['features']
    # _visual_Facet42 = dataset[visual_field_Facet42][segment]['features']
    # _visual_OpenFace1 = dataset[visual_field_OpenFace1][segment]['features']

    _acoustic_COVAREP = dataset[acoustic_field_COVAREP][segment]['features']
    # _acoustic_OpenSmile_EB10 = dataset[acoustic_field_OpenSmile_EB10][segment]['features']

    _wordvectors = dataset[text_field_WordVectors][segment]['features']

    # # Check if all modalities have the same number of elements (length of sequence)
    # if not (_words.shape[0] == _visual_Facet41.shape[0] == _visual_Facet42.shape[0] == _visual_OpenFace1.shape[0] == 
    #         _acoustic_COVAREP.shape[0] == _acoustic_OpenSmile_EB10.shape[0] == _wordvectors.shape[0]):
    #     print(f"Error: Inconsistent sequence lengths for segment {vid}")
    #     num_drop += 1
    #     continue  # Skip this segment and continue with the next one


     # Check if all modalities have the same number of elements (length of sequence)
    if not (_words.shape[0] == _visual_Facet41.shape[0] == _acoustic_COVAREP.shape[0]
                    == _wordvectors.shape[0]):
        print(f"Error: Inconsistent sequence lengths for segment {vid}")
        num_drop += 1
        continue  # Skip this segment and continue with the next one

    # Lists to hold the processed data for each modality
    words = []
    visual = []
    acoustic = []
    wordvectors = []

    # Remove speech pauses (um, uhh, etc.)
    for i, word in enumerate(_words):
        
        if word[0] != b'sp':  # Remove speech pauses
            words.append(word2id[word[0].decode('utf-8')])  # Decode bytes to string and add to words


            # Append visual features (check the shape of each feature)
            visual.append(_visual_Facet41[i, :])  # Facet41
            # visual.append(_visual_Facet42[i, :])  # Facet42
            # visual.append(_visual_OpenFace1[i, :])  # OpenFace1

            # Append acoustic features (check the shape of each feature)
            acoustic.append(_acoustic_COVAREP[i, :])  # COVAREP
            # acoustic.append(_acoustic_OpenSmile_EB10[i, :])  # OpenSmile_EB10

            # combined_acoustic = np.vstack((_acoustic_COVAREP[i, :], _acoustic_OpenSmile_EB10[i, :]))
            # acoustic.append(combined_acoustic)

            # Append word vectors
            wordvectors.append(_wordvectors[i, :])  # Word vectors


    # LOOK AT THE SHAPES

    # Check the shapes of the collected features before converting to numpy arrays
    # print(f"Word vectors shape: {np.asarray(wordvectors).shape}")
    print(f"Words shape: {np.asarray(words).shape}")
    print(f"WordVectors shape: {np.asarray(wordvectors).shape}")
    print(f"Acoustic shape: {np.asarray(acoustic).shape}")

    print(f"Visual shape: {np.asarray(visual).shape}")

    # Convert lists to numpy arrays
    words = np.asarray(words)
    visual = np.asarray(visual)
    acoustic = np.asarray(acoustic)
    wordvectors = np.asarray(wordvectors)

    # Z-normalization for visual modality (across all visual features)
    std_dev_visual = np.std(visual, axis=0, keepdims=True)
    visual = np.nan_to_num((visual - visual.mean(0, keepdims=True)) / (EPS + std_dev_visual))
    visual[:, std_dev_visual.flatten() == 0] = EPS  # Safeguard for zero standard deviation

    # Z-normalization for acoustic modality (across all acoustic features)
    acoustic_mean = np.nanmean(acoustic, axis=0, keepdims=True)
    std_dev_acoustic = np.nanstd(acoustic, axis=0, keepdims=True)
    std_dev_acoustic = np.nan_to_num(std_dev_acoustic)
    std_dev_acoustic[std_dev_acoustic == 0] = EPS  # Safeguard for zero standard deviation

    acoustic = np.nan_to_num((acoustic - acoustic_mean) / (EPS + std_dev_acoustic))

    # Z-normalization for word vectors
    wordvectors_mean = np.nanmean(wordvectors, axis=0, keepdims=True)
    std_dev_wordvectors = np.nanstd(wordvectors, axis=0, keepdims=True)
    std_dev_wordvectors = np.nan_to_num(std_dev_wordvectors)
    std_dev_wordvectors[std_dev_wordvectors == 0] = EPS  # Safeguard for zero standard deviation
    wordvectors = np.nan_to_num((wordvectors - wordvectors_mean) / (EPS + std_dev_wordvectors))

    # plot_hist2(wordvectors, acoustic)

    # Ensure no NaN or Inf values in the data
    if np.any(np.isnan(acoustic)) or np.any(np.isinf(acoustic)):
        print(f"Error in acoustic data for segment {vid}")
    if np.any(np.isnan(visual)) or np.any(np.isinf(visual)):
        print(f"Error in visual data for segment {vid}")
    if np.any(np.isnan(words)) or np.any(np.isinf(words)):
        print(f"Error in wordvectors data for segment {vid}")

    # Add the data to the appropriate split
    if vid in train_split:
        train.append(((words, visual, acoustic, wordvectors), label, segment))
    elif vid in dev_split:
        dev.append(((words, visual, acoustic, wordvectors), label, segment))
    elif vid in test_split:
        test.append(((words, visual, acoustic,wordvectors), label, segment))
    else:
        print(f"Found video that doesn't belong to any splits: {vid}")

# Output number of dropped datapoints
print(f"Total number of {num_drop} datapoints have been dropped.")
# Check how many words are in the vocabulary
vocab_size = len(word2id)
print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 5
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (19,)
WordVectors shape: (19, 300)
Acoustic shape: (19, 74)
Visual shape: (19, 47)
Words shape: (9,)
WordVectors shape: (9, 300)
Acoustic shape: (9, 74)
Visual shape: (9, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (4,)
WordVectors shape: (4, 300)
Acoustic shape: (4, 74)
Visual shape: (4, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Acoustic shape: (5, 74)
Visual shape: (5, 47)
Words shape: (10,)
WordVectors shape: (10, 300)
Acoustic shape: (10, 74)
Visual shape: (10, 47)
Words shape: (13,)
WordVectors shape: (13, 300)
Acoustic shape: (13, 74)
Visual shape: (13, 47)
Words shape: (7,)
WordVectors shape: (7, 300)
Acoustic shape: (7, 74)
Visual shape: (7, 47)
Words shape: (4,)
WordVectors shape: (4, 300)
Acoustic shape: (4, 74)
Visual shape: (4, 47)
Words shape: (5,)
WordVectors shape: (5, 300)
Aco

C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\numpy\lib\_nanfunctions_impl.py:1882: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
C:\Users\Viki\AppData\Local\Temp\ipykernel_19120\785341847.py:104: RuntimeWarning: invalid value encountered in subtract
  acoustic = np.nan_to_num((acoustic - acoustic_mean) / (EPS + std_dev_acoustic))


In [ ]:
# Ensure the train dataset has elements before sampling
if len(train) > 0:
    # Randomly sample one processed segment
    sample = random.choice(train)

    # Extract components
    (words, visual, acoustic, wordvectors), label, segment = sample

    # Display the details
    print(f"Segment: {segment}")
    print(f"Label: {label}")
    print(f"Words (sample, first 10 if too long): {words[:10] if len(words) > 10 else words}")
    print(f"Words shape: {words.shape}")
    print(f"Visual shape: {visual.shape}")
    print(f"Acoustic shape: {acoustic.shape}")
    print(f"WordVectors shape: {wordvectors.shape}")
else:
    print("The train dataset is empty. Check the preprocessing loop for issues.")


Segment: 8OtFthrtaJM[0]
Label: [[0.4]]
Words (sample, first 10 if too long): [129  46 179  42 125 720 721 245]
Words shape: (8,)
Visual shape: (8, 47)
Acoustic shape: (8, 74)
WordVectors shape: (8, 300)


In [ ]:
# turn off the word2id - define a named function here to allow for pickling
def return_unk():
    return UNK
word2id.default_factory = return_unk

In [ ]:
def multi_collate_acoustic(batch):
    '''
    Collate function for acoustic data only. Batch will be sorted based on the sequence length of the acoustic features.
    '''
    # Sort batch in descending order based on the length of the acoustic feature sequence
    batch = sorted(batch, key=lambda x: x[0][2].shape[0], reverse=True)
    
    # Extract labels and acoustic features from the batch
    labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0).float()
    acoustic = pad_sequence([torch.FloatTensor(sample[0][2]) for sample in batch], batch_first=True)
    
    # Sequence lengths (useful for RNNs)
    lengths = torch.LongTensor([sample[0][2].shape[0] for sample in batch])
    return acoustic, labels, lengths


batch_sz = 56
train_loader = DataLoader(train, shuffle=True, batch_size=batch_sz, collate_fn=multi_collate_acoustic)
dev_loader = DataLoader(dev, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate_acoustic)
test_loader = DataLoader(test, shuffle=False, batch_size=batch_sz*3, collate_fn=multi_collate_acoustic)

# let's create a temporary dataloader just to see how the batch looks like
temp_loader = iter(DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate_acoustic))
batch = next(temp_loader)

# print(batch[0].shape) # word vectors, padded to maxlen
print(batch[0].shape) # acoustic features
print(batch[1]) # labels
print(batch[2]) # lengths

torch.Size([8, 27, 74])
tensor([[ 0.2000],
        [-0.6000],
        [ 3.0000],
        [-0.4000],
        [ 2.2000],
        [ 2.0000],
        [ 0.6000],
        [-2.4000]])
tensor([27, 18, 12,  9,  7,  6,  4,  4])


In [ ]:
# from torch import optim
# from torch.utils.tensorboard import SummaryWriter
# 
# class SingleEncoderModelAudio(nn.Module):
#     def __init__(self, input_size, hidden_dim, num_layers, dropout_rate, output_size):
#         super(SingleEncoderModelAudio, self).__init__()
#         
#         self.input_size = input_size
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.dropout_rate = dropout_rate    
#         # self.num_categories = num_categories
#         self.output_size = output_size
#         # GRU layer
#         self.gru = nn.GRU(input_size=self.input_size, 
#                           hidden_size=self.hidden_dim,  # Correct parameter is hidden_size
#                           num_layers=self.num_layers, 
#                           batch_first=True,  # Input is [batch_size, seq_length, input_size]
#                           dropout=self.dropout_rate if self.num_layers > 1 else 0)
# 
#         # Fully connected output layer
#         # self.fc1 = nn.Linear((self.hidden_dim), num_categories)
#         self.fc2 = nn.Linear((self.hidden_dim), output_size)
# 
# 
#     
#     def forward(self, x, lengths):
#         batch_size = lengths.size(0)
# 
#         # GRU forward pass
#         _, hidden = self.gru(x)  # `hidden` shape: (num_layers, batch_size, hidden_dim)
#         hidden = hidden[-1]  # Use the last hidden state
#         
#         # Fully connected layers
#         # fc1_out = self.fc1(hidden)  # Output of shape [batch_size, num_categories]
#         output = self.fc2(hidden)  # Output of shape [batch_size, output_size]
#         
#         return output
# 
# 
#     def compute_loss(self, batch_pred, y_labels):
#         # Use Mean Squared Error loss for regression
#         loss_fn = nn.MSELoss()
#         loss = loss_fn(batch_pred, y_labels)
#         return loss
# 
# 
#     def create_optimizer(self, lr):
#         # Optimizer (Adam) with the specified learning rate
#         optimizer = optim.Adam(self.parameters(), lr=lr)
#         return optimizer
# 


In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter

class SingleEncoderModelAudio(nn.Module):
    def __init__(self, input_size, hidden_dim, num_layers, dropout_rate, output_size):
        super(SingleEncoderModelAudio, self).__init__()
        
        self.input_size = input_size
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout_rate = dropout_rate    
        self.output_size = output_size
        
        # GRU layer
        self.gru = nn.GRU(input_size=self.input_size, 
                          hidden_size=self.hidden_dim,  # Correct parameter is hidden_size
                          num_layers=self.num_layers, 
                          batch_first=True,  # Input is [batch_size, seq_length, input_size]
                          dropout=self.dropout_rate if self.num_layers > 1 else 0)

        # Fully connected output layer
        self.fc2 = nn.Linear(self.hidden_dim, output_size)
        
        # Tanh activation function to scale outputs between -3 and 3
        self.tanh = nn.Tanh()

    def forward(self, x, lengths):
        batch_size = lengths.size(0)

        # GRU forward pass
        _, hidden = self.gru(x)  # `hidden` shape: (num_layers, batch_size, hidden_dim)
        hidden = hidden[-1]  # Use the last hidden state
        
        # Fully connected layers
        output = self.fc2(hidden)  # Output of shape [batch_size, output_size]
        
        # Apply tanh and scale the output to [-3, 3]
        output = 3 * self.tanh(output)
        
        return output

    def compute_loss(self, batch_pred, y_labels):
        # Use Mean Squared Error loss for regression
        loss_fn = nn.MSELoss()
        loss = loss_fn(batch_pred, y_labels)
        return loss

    def create_optimizer(self, lr):
        # Optimizer (Adam) with the specified learning rate
        optimizer = optim.Adam(self.parameters(), lr=lr)
        return optimizer


In [ ]:



torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

CUDA = torch.cuda.is_available()
MAX_EPOCH = 1000

input_size = 74  # Example: Number of features per time step (adjust as per your data)
# hidden_sizes = int(input_size)  # Number of GRU units
hidden_sizes = 128
num_layers = 2  # Number of layers in GRU
num_categories = 7  # Number of categories for classification (adjust as needed)
output_size = 1  # Output size (e.g., 1 for binary classification)


fc1_size = hidden_sizes
dropout = 0.5
curr_patience = patience = 8
num_trials = 3
grad_clip_value = 1.0
weight_decay = 0.1

print("Model ???")
model = SingleEncoderModelAudio(input_size=input_size, 
                                hidden_dim=hidden_sizes,
                                num_layers=num_layers,
                                dropout_rate=dropout,
                                # num_categories=num_categories,
                                output_size=output_size)
print("Model created")

# model.embed.requires_grad = False
optimizer = model.create_optimizer(lr=0.001)
print("Optimizer created")


if CUDA:
    model.cuda()
    
criterion = nn.MSELoss(reduction='sum')
criterion_test = nn.MSELoss(reduction='sum')

best_valid_loss = float('inf')

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
lr_scheduler.step() # for some reason it seems the StepLR needs to be stepped once first

train_losses = []
valid_losses = []
for e in range(MAX_EPOCH):
    model.train()
    train_iter = tqdm_notebook(train_loader)
    train_loss = 0.0
    
    for batch in train_iter:
        model.zero_grad()
        a, y, l = batch
        batch_size = a.size(0)
        if CUDA:
            a = a.cuda()
            y = y.cuda()
            l = l.cuda()
        
        y_tilde = model(a, l)
        # print(f"batch_pred: {y_tilde.shape}")
        # print(f"labels: {y.shape}")
        
        loss = criterion(y_tilde, y)
        loss.backward()
        torch.nn.utils.clip_grad_value_([param for param in model.parameters() if param.requires_grad], grad_clip_value)
        optimizer.step()
        train_iter.set_description(f"Epoch {e}/{MAX_EPOCH}, current batch loss: {round(loss.item()/batch_size, 4)}")
        train_loss += loss.item()
    train_loss = train_loss / len(train)
    train_losses.append(train_loss)
    print(f"Training loss: {round(train_loss, 4)}")
    
    # print("LOOP COMPLETED")

    model.eval()
    with torch.no_grad():
        valid_loss = 0.0
        for batch in dev_loader:
            model.zero_grad()
            a, y, l = batch
            if CUDA:
                a = a.cuda()
                y = y.cuda()
                l = l.cuda()
            y_tilde = model(a, l)
            loss = criterion(y_tilde, y)
            valid_loss += loss.item()
    
    valid_loss = valid_loss/len(dev)
    valid_losses.append(valid_loss)
    print(f"Validation loss: {round(valid_loss, 4)}")
    print(f"Current patience: {curr_patience}, current trial: {num_trials}.")
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        print("Found new best model on dev set!")
        torch.save(model.state_dict(), 'modelaudio.std')
        torch.save(optimizer.state_dict(), 'optimaudio.std')
        curr_patience = patience
    else:
        curr_patience -= 1
        if curr_patience <= -1:
            print("Running out of patience, loading previous best model.")
            num_trials -= 1
            curr_patience = patience
            model.load_state_dict(torch.load('modelaudio.std'))
            optimizer.load_state_dict(torch.load('optimaudio.std'))
            lr_scheduler.step()
            print(f"Current learning rate: {optimizer.state_dict()['param_groups'][0]['lr']}")
    
    if num_trials <= 0:
        print("Running out of patience, early stopping.")
        break




Model ???
Model created
Optimizer created


C:\Users\Viki\Documents\Thesis\tryout3\.venv\lib\site-packages\torch\optim\lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
C:\Users\Viki\AppData\Local\Temp\ipykernel_19120\2342320971.py:50: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  train_iter = tqdm_notebook(train_loader)


  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 2.3361
Validation loss: 2.7291
Current patience: 8, current trial: 3.
Found new best model on dev set!


  0%|          | 0/23 [00:00<?, ?it/s]

...

Training loss: 2.2805
Validation loss: 2.7276
Current patience: 2, current trial: 1.


  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 2.2851
Validation loss: 2.7282
Current patience: 1, current trial: 1.


  0%|          | 0/23 [00:00<?, ?it/s]

Training loss: 2.2746
Validation loss: 2.728
Current patience: 0, current trial: 1.
Running out of patience, loading previous best model.
Current learning rate: 1e-05
Running out of patience, early stopping.


In [ ]:
# model.load_state_dict(torch.load('modelaudio.std'))
# y_true = []
# y_pred = []
# model.eval()
# with torch.no_grad():
#     test_loss = 0.0
#     for batch in test_loader:
#         model.zero_grad()
#         a, y, l = batch
#         if CUDA:
#             a = a.cuda()
#             y = y.cuda()
#             l = l.cuda()
#         y_tilde = model(a, l)
#         loss = criterion_test(y_tilde, y)
#         y_true.append(y_tilde.detach().cpu().numpy())
#         y_pred.append(y.detach().cpu().numpy())
#         test_loss += loss.item()
# print(f"Test set performance: {test_loss/len(test)}")
# y_true = np.concatenate(y_true, axis=0)
# y_pred = np.concatenate(y_pred, axis=0)
#                   
# y_true_bin = y_true >= 0
# y_pred_bin = y_pred >= 0
# bin_acc = accuracy_score(y_true_bin, y_pred_bin)
# print(f"Test set accuracy is {bin_acc}")

In [ ]:
# model.load_state_dict(torch.load('modelaudio.std'))
# print("Model loaded successfully!")
# 
# y_true = []
# y_pred = []
# model.eval()
# with torch.no_grad():
#     test_loss = 0.0
#     for batch in test_loader:
#         model.zero_grad()
#         a, y, l = batch
#         
#         # Ensure proper device placement (CUDA or CPU)
#         if torch.cuda.is_available():
#             a = a.cuda()
#             y = y.cuda()
#             l = l.cuda()
# 
#         # Print input shapes
#         print(f"Input shapes - a: {a.shape}, y: {y.shape}, l: {l.shape}")
# 
#         y_tilde = model(a, l)
# 
#         # Print model output shape
#         print(f"Output shape of the model: {y_tilde.shape}")
# 
#         loss = criterion_test(y_tilde, y)
# 
#         # Print loss for each batch
#         print(f"Batch loss: {loss.item()}")
# 
#         y_true.append(y_tilde.detach().cpu().numpy())
#         y_pred.append(y.detach().cpu().numpy())
#         test_loss += loss.item()
# 
#     print(f"Test set performance: {test_loss / len(test_loader)}")
# 
# # Concatenate true and predicted values
# y_true = np.concatenate(y_true, axis=0)
# y_pred = np.concatenate(y_pred, axis=0)
# 
# # Print the first few values before thresholding
# print(f"First few values of y_true: {y_true[:10]}")
# print(f"First few values of y_pred: {y_pred[:10]}")
# 
# y_true_bin = y_true >= 0
# y_pred_bin = y_pred >= 0
# 
# # Print binarized values
# print(f"Binarized y_true: {y_true_bin[:10]}")
# print(f"Binarized y_pred: {y_pred_bin[:10]}")
# 
# bin_acc = accuracy_score(y_true_bin, y_pred_bin)
# print(f"Test set accuracy is {bin_acc}")


In [ ]:
def convert_to_sentiment_category(score):
    if score == 3:
        return 'strongly positive'
    elif score >= 2 and score < 3:
        return 'positive'
    elif score >= 1 and score < 2:
        return 'weakly positive'
    elif score < 1 and score > -1:
        return 'neutral'
    elif score <= -1 and score > -2:
        return 'weakly negative'
    elif score <= -2 and score > -3:
        return 'negative'
    elif score == -3:
        return 'strongly negative'
    else:
        return 'unknown'


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score


# Ensure that the model is loaded
model.load_state_dict(torch.load('modelaudio.std'))
print("Model loaded successfully!")

y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    test_loss = 0.0
    for batch in test_loader:
        model.zero_grad()
        a, y, l = batch
        
        # Ensure proper device placement (CUDA or CPU)
        if torch.cuda.is_available():
            a = a.cuda()
            y = y.cuda()
            l = l.cuda()

        # Print input shapes
        print(f"Input shapes - a: {a.shape}, y: {y.shape}, l: {l.shape}")

        y_tilde = model(a, l)

        # Print model output shape
        print(f"Output shape of the model: {y_tilde.shape}")

        loss = criterion_test(y_tilde, y)

        # Print loss for each batch
        print(f"Batch loss: {loss.item()}")

        y_true.append(y.detach().cpu().numpy())  # True values
        y_pred.append(y_tilde.detach().cpu().numpy())  # Predicted values
        test_loss += loss.item()

    print(f"Test set performance (total loss): {test_loss / len(test_loader)}")

# Concatenate true and predicted values
y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

# Convert both y_true and y_pred to sentiment categories
y_true_categories = [convert_to_sentiment_category(score) for score in y_true]
y_pred_categories = [convert_to_sentiment_category(score) for score in y_pred]

print(f"First few values of y_true: {y_true[:10]}")
print(f"First few true sentiment labels: {y_true_categories[:10]}")

print(f"First few values of y_pred: {y_pred[:10]}")
print(f"First few predicted sentiment labels: {y_pred_categories[:10]}")

# Compute accuracy by comparing predicted categories with true categories
accuracy = accuracy_score(y_true_categories, y_pred_categories)
print(f"Accuracy: {accuracy}")


Model loaded successfully!
Input shapes - a: torch.Size([168, 44, 74]), y: torch.Size([168, 1]), l: torch.Size([168])
Output shape of the model: torch.Size([168, 1])
Batch loss: 609.8631591796875
Input shapes - a: torch.Size([168, 82, 74]), y: torch.Size([168, 1]), l: torch.Size([168])
Output shape of the model: torch.Size([168, 1])
Batch loss: 386.1199645996094
Input shapes - a: torch.Size([168, 112, 74]), y: torch.Size([168, 1]), l: torch.Size([168])
Output shape of the model: torch.Size([168, 1])
Batch loss: 582.5965576171875
Input shapes - a: torch.Size([168, 58, 74]), y: torch.Size([168, 1]), l: torch.Size([168])
Output shape of the model: torch.Size([168, 1])
Batch loss: 368.5628662109375
Input shapes - a: torch.Size([14, 21, 74]), y: torch.Size([14, 1]), l: torch.Size([14])
Output shape of the model: torch.Size([14, 1])
Batch loss: 12.608111381530762
Test set performance (total loss): 391.9501317977905
First few values of y_true: [[ 0.2]
 [ 2. ]
 [ 0.2]
 [ 0.8]
 [-1. ]
 [ 0.2]
 

C:\Users\Viki\AppData\Local\Temp\ipykernel_19120\262063581.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modelaudio.std'))
